In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen
from tqdm import tqdm_notebook
import unidecode

# Crawling Pitchfork Review Rating

In [5]:
def pf_rating(starting, ending):

    df = pd.DataFrame(columns=['artists', 'albums','genre', 'release_date', 'ratings'])

    for j in tqdm_notebook(range(starting, ending, 12)):

        url = 'https://pitchfork.com/api/v2/search/?types=reviews&hierarchy=sections%2Freviews%2Falbums&sort=publishdate%20desc%2Cposition%20asc&size=12&start=' + str(j)
        data = json.load(urlopen(url))

        for i in range(len(data['results']['list'])):

            try:
                artist_name = data['results']['list'][i]['artists'][0]['display_name']
            except IndexError:
                artist_name = "Various Artist"

            album_name = data['results']['list'][i]['tombstone']['albums'][0]['album']['display_name']
            rating_score = data['results']['list'][i]['tombstone']['albums'][0]['rating']['rating']
            release_date = data['results']['list'][i]['pubDate'][:-14]

            genre_ls = []
            for k in range(len(data['results']['list'][i]['genres'])):
                genre_k = data['results']['list'][i]['genres'][k]['display_name']
                genre_ls.append(genre_k)



            input_data={
                'artists' : artist_name,
                'albums' : album_name,
                'genre' : genre_ls,
                'release_date' : release_date,
                'ratings' : rating_score,
            }

            df.loc[len(df)] = input_data
    
    return df

In [ ]:
df1 = pf_rating(0, 2400)

In [23]:
df2 = pf_rating(2401, 4800)

HBox(children=(IntProgress(value=0, max=200), HTML(value='')))

In [21]:
df3 = pf_rating(4801, 6000)

HBox(children=(IntProgress(value=0), HTML(value='')))

In [22]:
df4 = pf_rating(6001, 8400)

HBox(children=(IntProgress(value=0, max=200), HTML(value='')))

In [24]:
df5 = pf_rating(8401, 8772)

HBox(children=(IntProgress(value=0, max=31), HTML(value='')))

In [29]:
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [36]:
df = df[:8748]

In [38]:
df.shape

(8748, 5)

In [39]:
df.to_csv("../data/pitchfork_rating_1118.csv", index=False)

In [17]:
df = pd.read_csv("../data/rating/rating_pitchfork-1118-utf8.csv")

In [16]:
def decodeuni(str):
    return unidecode.unidecode(str)

In [13]:
df['artists'] = df['artists'].apply(lambda x : str(x))
df['albums'] = df['albums'].apply(lambda x : str(x))
df['artists'] = df['artists'].apply(decodeuni)
df['albums'] = df['albums'].apply(decodeuni)

In [14]:
df.to_csv("../data/rating_pitchfork-1118.csv", index=False)

In [7]:
df = pd.read_csv("../data/rating/rating_pitchfork-1118-utf8.csv")

In [15]:
df.tail()

,index,artist,album,release_date,ratings
8742,8742,Solar Bears,She Was Coloured In,03/01/2011,6.9
8743,8743,Big Troubles,Worry,03/01/2011,7.5
8744,8744,Jason Forrest,Utopia EP,03/01/2011,6.5
8745,8745,Shackleton,Fabric 55,03/01/2011,7.7
8746,8746,Ghostface Killah,Apollo Kids,03/01/2011,7.3


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8747 entries, 0 to 8746
Data columns (total 5 columns):
index           8747 non-null int64
artist          8746 non-null object
album           8746 non-null object
release_date    8747 non-null object
ratings         8747 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 341.8+ KB


### Upload to MySQL database

In [18]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [19]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [20]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [22]:
df.to_sql(name="rating_pitchFork_copy", con=engine, if_exists='replace')